In [1]:
import datetime

import backtrader as bt


In [2]:
"""
A long only strategy: buy if close is greate than SMA; sell if the close is smaller than the SMA.
"""
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 10),
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f"{dt.isoformat()}, {txt}")
    
    def log_order(self, order: bt.order.Order):
        direction = 'BUY' if order.isbuy() else 'SELL'
        self.log(f"{direction} EXECUTED, Size: {order.executed.size}, Price: {order.executed.price:.2f}, Cost: {order.executed.value}, Comm: {order.executed.comm:.2f}")

    def __init__(self):
        self.dataclose = self.datas[0].close

        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(self.datas[0], period=self.params.maperiod)

        # Indicators for plotting only
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25, subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order: bt.order.Order):
        if order.status in [order.Submitted, order.Accepted]:
            return
        
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log_order(order)
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            elif order.issell():
                self.log_order(order)
            
            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log("Order Canceled/Margin/Rejected")
        
        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade: bt.trade.Trade):
        if not trade.isclosed:
            return
        self.log(f"OPERATION PROFIT, Gross {trade.pnl:.2f}, Net {trade.pnlcomm:.2f}")

    def next(self):
        self.log(f'Close Price: {self.dataclose[0]:.2f}')

        # if there is a pending order, we will not send a 2nd one
        if self.order:
            return
    
        # check if we are in the market
        if not self.position:
            if self.dataclose[0] > self.sma[0]:
                self.log(f"BUY CREATE, {self.dataclose[0]:.2f}")
                self.order = self.buy()
        
        else:
            if self.dataclose[0] < self.sma[0]:
                self.log(f"SELL CREATE, {self.dataclose[0]:.2f}")
                self.order = self.sell()


In [18]:
StoreCls = bt.stores.OandaStore
DataCls = bt.feeds.OandaData
BrokerCls = bt.brokers.OandaBroker

class args:
    token = None
    account = None
    live = False
    
    timeframe = 'Minutes' # bt.TimeFrame.Names
    timeframe1 = None # TimeFrame for Resample/Replay - Data1
    compression = 1 # Compression for Resample/Replay
    compression1 = None # Compression for Resample/Replay - Data1
    replay = None
    resample = None

    no_store = False # Do not use the store pattern
    broker = True # Use Oanda as broker



In [15]:
cerebro = bt.Cerebro()

storekwargs = dict(
    token=args.token,
    account=args.account,
    practice=not args.live,
)

store = StoreCls(**storekwargs) if not args.no_store else None
if args.broker:
    broker = BrokerCls(**storekwargs) if args.no_store else store.getbroker()
    cerebro.setbroker(broker)

timeframe = bt.TimeFrame.TFrame(args.timeframe)

# Manage data1 parameters
tf1 = args.timeframe1 # TimeFrame for Resample/Replay
tf1 = bt.TimeFrame.TFrame(tf1) if tf1 is not None else timeframe
cp1 = args.compression1
cp1 = cp1 if cp1 is not None else args.compression

if args.resample or args.replay:
    datatf = datatf1 = bt.TimeFrame.Ticks
    datacomp = datacomp1 = 1
else:
    datatf = timeframe
    datacomp = args.compression
    datatf1 = tf1
    datacomp1 = cp1

fromdate = None

DataFactory = DataCls if args.no_store else store.getdata



In [25]:
DataCls

backtrader.feeds.oanda.OandaData

In [26]:
store.getdata

<bound method OandaStore.getdata of <class 'backtrader.stores.oandastore.OandaStore'>>

In [ ]:

fromdate = None
if args.fromdate:
    dtformat = '%Y-%m-%d' + ('T%H:%M:%S' * ('T' in args.fromdate))
    fromdate = datetime.datetime.strptime(args.fromdate, dtformat)

DataFactory = DataCls if args.no_store else store.getdata

datakwargs = dict(
    timeframe=datatf, compression=datacomp,
    qcheck=args.qcheck,
    historical=args.historical,
    fromdate=fromdate,
    bidask=args.bidask,
    useask=args.useask,
    backfill_start=not args.no_backfill_start,
    backfill=not args.no_backfill,
    tz=args.timezone
)

if args.no_store and not args.broker:   # neither store nor broker
    datakwargs.update(storekwargs)  # pass the store args over the data

data0 = DataFactory(dataname=args.data0, **datakwargs)

data1 = None
if args.data1 is not None:
    if args.data1 != args.data0:
        datakwargs['timeframe'] = datatf1
        datakwargs['compression'] = datacomp1
        data1 = DataFactory(dataname=args.data1, **datakwargs)
    else:
        data1 = data0

rekwargs = dict(
    timeframe=timeframe, compression=args.compression,
    bar2edge=not args.no_bar2edge,
    adjbartime=not args.no_adjbartime,
    rightedge=not args.no_rightedge,
    takelate=not args.no_takelate,
)

if args.replay:
    cerebro.replaydata(data0, **rekwargs)

    if data1 is not None:
        rekwargs['timeframe'] = tf1
        rekwargs['compression'] = cp1
        cerebro.replaydata(data1, **rekwargs)

elif args.resample:
    cerebro.resampledata(data0, **rekwargs)

    if data1 is not None:
        rekwargs['timeframe'] = tf1
        rekwargs['compression'] = cp1
        cerebro.resampledata(data1, **rekwargs)

else:
    cerebro.adddata(data0)
    if data1 is not None:
        cerebro.adddata(data1)

if args.valid is None:
    valid = None
else:
    valid = datetime.timedelta(seconds=args.valid)
# Add the strategy
cerebro.addstrategy(TestStrategy,
                    smaperiod=args.smaperiod,
                    trade=args.trade,
                    exectype=bt.Order.ExecType(args.exectype),
                    stake=args.stake,
                    stopafter=args.stopafter,
                    valid=valid,
                    cancel=args.cancel,
                    donotcounter=args.donotcounter,
                    sell=args.sell,
                    usebracket=args.usebracket)

# Live data ... avoid long data accumulation by switching to "exactbars"
cerebro.run(exactbars=args.exactbars)
if args.exactbars < 1:  # plotting is possible
    if args.plot:
        pkwargs = dict(style='line')
        if args.plot is not True:  # evals to True but is not True
            npkwargs = eval('dict(' + args.plot + ')')  # args were passed
            pkwargs.update(npkwargs)

        cerebro.plot(**pkwargs)